# Reading Files

In [76]:
from os import listdir
import xml.etree.ElementTree as ET

class BoardGame:
    def __init__(self, root):
        self.id = root.attrib["id"].strip()
        nameTags = root.findall("name")
        for n in nameTags:
            if n.attrib["type"] == "primary":
                self.name = n.attrib["value"]
        self.description = ""
        if root.find("description").text != None:
            self.description = root.find("description").text.strip()
        self.year = 0
        if len(root.find("yearpublished").attrib["value"].strip()) != 0:
            self.year = int(root.find("yearpublished").attrib["value"])
        minPlayers = 0
        if len(root.find("minplayers").attrib["value"].strip()) != 0:
            minPlayers = int(root.find("minplayers").attrib["value"])
        maxPlayers = 0
        if len(root.find("maxplayers").attrib["value"].strip()) != 0:
            maxPlayers = int(root.find("maxplayers").attrib["value"])
        self.numPlayer = (minPlayers, maxPlayers)
        self.playTime = 0
        if len(root.find("playingtime").attrib["value"]) != 0:
            self.playTime = int(root.find("playingtime").attrib["value"])
        self.minAge = 0
        if len(root.find("minage").attrib["value"]) != 0:
            self.minAge = int(root.find("minage").attrib["value"])
        self.mechanics = []
        self.family = []
        self.category = []
        self.implementations = []
        xmlLinkLists = root.findall("link")
        for l in xmlLinkLists:
            if l.attrib["type"] == "boardgamemechanic":
                self.mechanics.append((l.attrib["id"].strip(), l.attrib["value"].strip()))
            if l.attrib["type"] == "boardgamecategory":
                self.category.append((l.attrib["id"].strip(), l.attrib["value"].strip()))
            if l.attrib["type"] == "boardgamefamily":
                self.family.append((l.attrib["id"].strip(), l.attrib["value"].strip()))
            if l.attrib["type"] == "boardgameimplementation":
                self.implementations.append(l.attrib["id"].strip())
        self.usersRating = int(root.find("statistics")[0].find("usersrated").attrib["value"])
        self.usersOwned = int(root.find("statistics")[0].find("owned").attrib["value"])
        self.avgRating = float(root.find("statistics")[0].find("average").attrib["value"])
        self.medRating = float(root.find("statistics")[0].find("median").attrib["value"])
        self.stdRating = float(root.find("statistics")[0].find("stddev").attrib["value"])
        self.rankList = []
        xmlRankLists = root.find("statistics")[0].find("ranks")
        for r in xmlRankLists:
            self.rankList.append(r.attrib["friendlyname"].strip())
    def __str__(self):
        return "[" + self.name + "] (" + str(self.year) + ") players:" + str(self.numPlayer) + \
            " time: " + str(self.playTime) + " rating: " + str(self.avgRating)
xmlFiles = [f for f in listdir("GamesXML/") if ".xml" in f]

boardgameDict = {}
mechanicsDict = {}
familyDict = {}
categoryDict = {}
for f in xmlFiles:
    tree = ET.parse("GamesXML/" + f)
    root = tree.getroot()
    for item in root:
        boardgame = BoardGame(item)
        boardgameDict[boardgame.id] = boardgame
        for m in boardgame.mechanics:
            mechanics[m[0]] = m[1]
        for f in boardgame.family:
            family[f[0]] = f[1]
        for c in boardgame.category:
            category[c[0]] = c[1]

In [77]:
def filterUniqueGames(currentList):
    newList = []
    for g in currentList:
        add = True
        for id in g.implementations:
            if id in newDict:
                add = False
                break
        if add:
            newList.append(g)
    return newList

def filterGamesWithMinUsers(currentList, users):
    newList = []
    for g in currentList:
        if g.usersRating > users or g.usersOwned > users:
            newList.append(g)
    return newList

def filterGamesWithYear(currentList, minYear=-1, maxYear=-1):
    newList = []
    for g in currentList:
        if (minYear <= 0 or g.year >= minYear) and (maxYear <= 0 or g.year <= maxYear):
            newList.append(g)
    return newList

def filterGamesWithName(currentList, name):
    newList = []
    for g in currentList:
        if name.strip().lower() in g.name.strip().lower():
            newList.append(g)
    return newList

In [82]:
currentList = boardgameDict.values()
# currentList = filterGamesWithYear(currentList, 2019)
currentList = filterGamesWithName(currentList, "dog")
for g in currentList:
    print(g)

[A Dog's World] (2015) players:(5, 0) time: 60 rating: 0.0
[Wardogs] (2008) players:(0, 0) time: 180 rating: 6.8
[Dogs In The Yard] (2015) players:(6, 0) time: 60 rating: 5.83333
[Dirty Dog] (0) players:(8, 0) time: 0 rating: 6.0
[Reservoir Dogs] (2012) players:(6, 0) time: 99 rating: 0.0
[Dogfight!] (1971) players:(2, 0) time: 45 rating: 5.0
[Doge] (2000) players:(4, 0) time: 45 rating: 6.51596
[Dogfight] (2008) players:(4, 0) time: 60 rating: 0.0
[Dog Pound] (1999) players:(5, 0) time: 20 rating: 5.0
[Dogma the Board Game] (2007) players:(2, 0) time: 0 rating: 0.0
[Dogfight squadron cards] (2018) players:(2, 0) time: 60 rating: 0.0
[ペーターと2匹の牧羊犬 (Peter's Two Sheep Dogs)] (2018) players:(2, 0) time: 30 rating: 6.9
[Monopoly: Snoopy It's A Dog's Life] (2005) players:(6, 0) time: 120 rating: 4.14286
[Do you look like your dog?] (2007) players:(4, 0) time: 0 rating: 2.83333
[DOG] (1997) players:(6, 0) time: 45 rating: 6.37587
[Scooby-doo! Get That Dog] (1999) players:(4, 0) time: 20 ratin